In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import datetime
import re

In [3]:
def get_next_page(driver):
    try:
        driver.find_element_by_class_name("ui_button.nav.next.primary").send_keys(Keys.ENTER)
        return True
    except:
        print('마지막 페이지입니다')
        return False

In [4]:
def convert_date(date):
    stay_date = date.split('숙박 날짜: ')
    stay_date = stay_date[1]

    # 20XX년 X월 형태의 str
    stay_date = stay_date.replace('년', '').replace(' ', '').replace('월', '')
    dt = datetime.datetime.strptime(stay_date, '%Y%m')
    dt = dt.date() #20XX-XX-01 형태의 date 객체
    return dt

In [23]:
# convert_date 실행 예시
convert_date('숙박 날짜: 2020년 11월')

datetime.date(2020, 11, 1)

In [5]:
def get_data(driver):
    iteration = True
    df = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])
    try:
        time.sleep(20)
        #WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 
        #                                                                    "div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu")))
        # 더보기 클릭
        driver.find_element_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu").click()

        time.sleep(5)
        # 숙박 날짜 가져오기
        dates = driver.find_elements_by_class_name("_34Xs-BQm")
        stay_dates = []
        for d in dates:
            stay_date = convert_date(d.text)
            stay_dates.append(stay_date)
            # 날짜 수정 - 코로나 전 1년 반 - 코로나 후 1년 반
            if stay_date < datetime.date(2018,6,1): 
                iteration = False

        # 닉네임 가져오기
        nicknames = driver.find_elements_by_css_selector("div._310S4sqz > div > div._2fxQ4TOx > span > a")
        # 별점 가져오기
        rating_codes = driver.find_elements_by_css_selector("div.oETBfkHU > div._2UEC-y30 > div > span")
        scores = []
        for rc in rating_codes:
            cls_attr = str(rc.get_attribute("class")).split("ui_bubble_rating bubble_")
            score = int(cls_attr[1])
            scores.append(score)

        # 리뷰 가져오기
        reviews = driver.find_elements_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu > div.cPQsENeY > q > span")
        
        for i in range(len(nicknames)):
            df.loc[i] = [nicknames[i].text, scores[i], reviews[i].text, stay_dates[i]]
            #print(df.loc[i])

    except ElementClickInterceptedException as e:
        print(e)
    except StaleElementReferenceException as e:
        print(e)

    finally:
        return df, iteration

In [6]:
def hotel_review_crawling(url):
    # 데이터 수집
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(options=options)

    data = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])

    #첫페이지 시작
    #데이터 크롤링, df에 추가하기
    #다음페이지 클릭 -> disbaled 나오기 전까지 반복

    driver.get(url)
    driver.implicitly_wait(10)
    iteration = True
    page = 1
    while iteration:
        print(page, '번째 페이지')
        tmp_df, iteration = get_data(driver)
        data = pd.concat([data, tmp_df], axis=0, ignore_index=True)
        if iteration == False:
            break
        iteration = get_next_page(driver) 
        page += 1

    #data.to_csv('./hotel_review_test_sillastay_mapo.csv', encoding="utf-8-sig")
    
    return data

In [7]:
def preprocessing_data(data):
    # 개행문자 제거
    for idx, review in enumerate(data['review']):
        data['review'][idx] = review.replace('\n', ' ')
    # 날짜순으로 내림차순 정렬
    data.sort_values(by='stay_date', ascending=False, 
                     ignore_index=True, inplace=True)
    # 날짜 수정 - 코로나 전 1년 반 - 코로나 후 1년 반
    data = data[data['stay_date'] > datetime.date(2018,5,1)]
    # 특수문자, 이모지 제거
    ###
    return data

In [8]:
# 실제로 실행하는 코드
url = "https://www.tripadvisor.co.kr/Hotel_Review-g294197-d19972497-Reviews-Shilla_Stay_Samsung-Seoul.html"

data_sample = hotel_review_crawling(url)
data_sample

1 번째 페이지
2 번째 페이지
3 번째 페이지
4 번째 페이지
5 번째 페이지
6 번째 페이지
7 번째 페이지
8 번째 페이지
9 번째 페이지
10 번째 페이지
11 번째 페이지
12 번째 페이지
13 번째 페이지
14 번째 페이지
15 번째 페이지
16 번째 페이지
17 번째 페이지
18 번째 페이지
19 번째 페이지
20 번째 페이지
21 번째 페이지
22 번째 페이지
23 번째 페이지
24 번째 페이지
25 번째 페이지
마지막 페이지입니다


,nickname,score,review,stay_date
0,오리6,50,호캉스하러 왔는데 지어진지 얼마되지 않아 시설도 깨끗하고 직원분들도 너무 친절하고 ...,2021-07-01
1,Sunny Lee,50,깨끗하고 편안한 시설과 친절한 서비스에 너무 만족합니다 앞으로도 또 찾고 싶은 가성...,2021-07-01
2,초코초코촠,50,접근성도 좋고 침구도 안락합니다 위생도 괜찮구요 룸서비스가 없는건 다소 아쉽습니다 ...,2021-07-01
3,쑥스런,40,우선 체크인 시간이 너무 길었어요 ㅠ 체크인 하면서 대기하는 라운지가 잘 되어있기는...,2021-07-01
4,Kang,50,일단 뷰가 신라스테이 중 제일 조아보여서예약했어요.\n커플 호캉스 즐기러왔습니다!!...,2021-06-01
...,...,...,...,...
118,Kim,50,오픈한지 얼마 안 돼서 깔끔하고\n로비 및 루프탑에서 바라보는 코엑스와 종합운동장은...,2020-05-01
119,Rick,50,신축이라 최고로 좋습니다!! 외관도 훌륭하고 특히 라운지가 좋습니다| 야외 라운지에...,2020-05-01
120,엉덩이흔드록바,50,이거뭐.. 솔직히 3성이라고 하는데 다른 특급호텔들 보다 직원들 서비스가 너무 좋아...,2020-05-01
121,Lee,50,넘 조어요. 역시 싱라스테이 믿고 갑니다. 비즈니스 호텔이지만 특급 사비스에 감동 ...,2020-05-01


In [9]:
# 기본 전처리 전 원본 데이터 저장(유실방지)
data_sample.to_csv('./data/hotel_review_shillastay_samsung_origin.csv', encoding="utf-8-sig")

In [11]:
data = preprocessing_data(data_sample)

In [12]:
data

,nickname,score,review,stay_date
0,오리6,50,호캉스하러 왔는데 지어진지 얼마되지 않아 시설도 깨끗하고 직원분들도 너무 친절하고 ...,2021-07-01
1,초코초코촠,50,접근성도 좋고 침구도 안락합니다 위생도 괜찮구요 룸서비스가 없는건 다소 아쉽습니다 ...,2021-07-01
2,쑥스런,40,우선 체크인 시간이 너무 길었어요 ㅠ 체크인 하면서 대기하는 라운지가 잘 되어있기는...,2021-07-01
3,Sunny Lee,50,깨끗하고 편안한 시설과 친절한 서비스에 너무 만족합니다 앞으로도 또 찾고 싶은 가성...,2021-07-01
4,Kang,50,일단 뷰가 신라스테이 중 제일 조아보여서예약했어요. 커플 호캉스 즐기러왔습니다!! ...,2021-06-01
...,...,...,...,...
118,woorams,50,"신라스테이 삼성점 가족들과 방문했습니다! 저녁에 뷔페에 방문해 스테이크, 엘에이 갈...",2020-05-01
119,Jennifer,50,혼캉스로 신라스테이 컴포트레저룸 예약해서왔는데 뷰도 너무 힐링되고 좋아요~ 모션베드...,2020-05-01
120,양지은,50,"남자친구랑 어디 호캉스 갈까 하다가 삼성점이 오픈했다고 해서 방문했는데, 잘한 선택...",2020-05-01
121,정영훈,50,"평소 신라스테이 구로, 서초점을 이용하다 삼성점 오픈이후 처음 이용했는데 시설도 청...",2020-04-01


In [13]:
# csv 저장을 위한 date to str convert
data['stay_date'] = data['stay_date'].apply(lambda X : X.strftime('%Y-%m-%d'))
# 기본 전처리후 데이터 csv 파일로 저장 
#data.to_csv('./data/hotel_review_shillastay_yeoksam.csv', encoding="utf-8-sig")

In [14]:
# 리뷰 내 이모지 제거
data['review'] = data['review'].apply(lambda X : X.encode('euc-kr', 'ignore').decode('euc-kr'))
#data.to_csv('./data/hotel_review_shillastay_yeoksam_del_emoji.csv', encoding="utf-8-sig")

In [15]:
# 리뷰 내 특수문자 제거
test_data = data
special = re.compile(r'[^ A-Za-z0-9가-힣+]')
test_data['review'] = [special.sub('', s) for s in test_data['review']]

# 특수문자 제거 후 파일 저장
test_data.to_csv('./data/hotel_review_shillastay_samsung_final.csv', encoding="utf-8-sig")

In [16]:
test_data

,nickname,score,review,stay_date
0,오리6,50,호캉스하러 왔는데 지어진지 얼마되지 않아 시설도 깨끗하고 직원분들도 너무 친절하고 ...,2021-07-01
1,초코초코촠,50,접근성도 좋고 침구도 안락합니다 위생도 괜찮구요 룸서비스가 없는건 다소 아쉽습니다 ...,2021-07-01
2,쑥스런,40,우선 체크인 시간이 너무 길었어요 체크인 하면서 대기하는 라운지가 잘 되어있기는 ...,2021-07-01
3,Sunny Lee,50,깨끗하고 편안한 시설과 친절한 서비스에 너무 만족합니다 앞으로도 또 찾고 싶은 가성...,2021-07-01
4,Kang,50,일단 뷰가 신라스테이 중 제일 조아보여서예약했어요 커플 호캉스 즐기러왔습니다 루프탑...,2021-06-01
...,...,...,...,...
118,woorams,50,신라스테이 삼성점 가족들과 방문했습니다 저녁에 뷔페에 방문해 스테이크 엘에이 갈비 ...,2020-05-01
119,Jennifer,50,혼캉스로 신라스테이 컴포트레저룸 예약해서왔는데 뷰도 너무 힐링되고 좋아요 모션베드도...,2020-05-01
120,양지은,50,남자친구랑 어디 호캉스 갈까 하다가 삼성점이 오픈했다고 해서 방문했는데 잘한 선택인...,2020-05-01
121,정영훈,50,평소 신라스테이 구로 서초점을 이용하다 삼성점 오픈이후 처음 이용했는데 시설도 청결...,2020-04-01
